In [1]:
%run 1._Add_Psilocybin_to_Coynebacterium_Model.ipynb #change model source here

No objective coefficients in model. Unclear what should be optimized


In [2]:
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
target = model.reactions.EX_psi_e

# Finding the Bottleneck


In [4._Production_Rate_Overview.ipynb](4._Production_Rate_Overview.ipynb), it was found that the production of tryptophan, TRPS1, had a low flux.

In [5]:
model.reactions.TRPS1 #our limiting factor

Reaction identifier,TRPS1
Name,tryptophan synthase (indoleglycerol phosphate)
Memory address,0x022ef2025978
Stoichiometry,3ig3p_c + ser_L_c --> g3p_c + h2o_c + trp_L_c C'-(3-Indolyl)-glycerol 3-phosphate + L_Serine --> Glyceraldehyde 3-phosphate + H2O + L_Tryptophan
GPR,cg3363 and cg3364
Lower bound,0.0
Upper bound,0.22043


In [6]:
with model: #input 1 of 2
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("TRPS1 Flux [mmol gDW^-1 h^-1]:",model.reactions.TRPS1.flux)

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPS1 Flux [mmol gDW^-1 h^-1]: 5.69299883708285e-05


## First metabolite, C'-(3-Indolyl)-glycerol 3-phosphate

In [7]:
with model: #input 1 of 2
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('3ig3p_c','reaction'):
        print("%s : %s" % (x.id, x.flux))    

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
IGPS : 0.09063538000775301
TRPS1 : 5.69299883708285e-05
TRPS3 : 0.09057845001938218
INPSHE : 0.0


# Second metabolite, L_Serine

In [8]:
with model: #input 2 of 2
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('ser_L_c','reaction'):
        print("%s:flux %s" % (x.id, x.flux))    

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
SERAT:flux 0.000113859976741657
GPDDA3:flux 0.0
GHMT2r:flux 0.00022771995348257009
PSERT:flux 0.3892699999999998
PSP_L:flux 0.3892699999999998
SERD_L:flux 0.3888714900814047
PSERtex:flux 0.0
SERabcpp:flux 0.0
SERt3pp:flux 0.0
SERTRS:flux 0.0
SERTRS2:flux 0.0
TRPS1:flux 5.69299883708285e-05
TRPS2:flux 0.0
PROTEIN:flux 0.0


In [9]:
with model:
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('ser_L_c','reaction'):
        print("%s: %s" % (x.id, x.reaction))

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
SERAT: accoa_c + ser_L_c --> acser_c + coa_c
GPDDA3: g3ps_c + h2o_c --> glyc3p_c + h_c + ser_L_c
GHMT2r: ser_L_c + thf_c <=> gly_c + h2o_c + mlthf_c
PSERT: 3php_c + glu_L_c --> akg_c + pser_L_c
PSP_L: h2o_c + pser_L_c --> pi_c + ser_L_c
SERD_L: ser_L_c --> nh4_c + pyr_c
PSERtex: pser_L_e --> pser_L_c
SERabcpp: atp_c + h2o_c + ser_L_e --> adp_c + h_c + pi_c + ser_L_c
SERt3pp: h_e + ser_L_c --> h_c + ser_L_e
SERTRS: atp_c + ser_L_c + trnaser_c --> amp_c + ppi_c + sertrna_c
SERTRS2: atp_c + ser_L_c + trnasecys_c --> amp_c + ppi_c + sertrna_c
TRPS1: 3ig3p_c + ser_L_c --> g3p_c + h2o_c + trp_L_c
TRPS2: indole_c + ser_L_c --> h2o_c + trp_L_c
PROTEIN: 1.3527 ala_L_c + 0.38519 arg_L_c + 0.39281 asn_L_c + 0.39281 asp_L_c + 40.0656 atp_c + 0.089437 cys_L_c + 0.69314 gln_L_c + 1.1139 glu_L_c + 0.71651 gly_c + 30.7601 h2o_c + 0.13619 his_L_c + 0.38316 ile_L_c + 0.71347 leu_L_c + 0.37909 lys_L_c + 0.15347 met_L_c + 0.26018 phe_L_c + 0.323

PSERT, PSP_L, and SERD_L seem to be the relevant reactions limiting TRPS1 (tryptophan production)

In [10]:
model.reactions.PSERT.reaction

'3php_c + glu_L_c --> akg_c + pser_L_c'

In [11]:
model.reactions.PSP_L.reaction

'h2o_c + pser_L_c --> pi_c + ser_L_c'

These two reactions lead to the production of Serine

In [12]:
model.reactions.SERD_L.reaction

'ser_L_c --> nh4_c + pyr_c'

This reaction breaks it down, leaving just enough for other reactions.

In [13]:
with model:
    model.reactions.SERD_L.upper_bound=0
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. psi production [mmol gDW^-1 h^-1]:", psi_production)
    print("Theoretical max. yield [mmol-psi / mmol-gl]:", psi_production / (-1*model.reactions.EX_glc_e.flux))

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
Theoretical max. yield [mmol-psi / mmol-gl]: 1.3830396904616058e-05


#### reduce this breakdown to nearly 0, effectively knocking it out/heavily downregulating

In [14]:
with model:
    model.reactions.SERD_L.bounds=0.01,0.01
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('ser_L_c','reaction'):
        print("%s:flux %s" % (x.id, x.flux))

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
SERAT:flux 0.00011385997674168147
GPDDA3:flux 0.0
GHMT2r:flux 0.3790992100348873
PSERT:flux 0.3892699999999998
PSP_L:flux 0.3892699999999998
SERD_L:flux 0.01
PSERtex:flux 0.0
SERabcpp:flux 0.0
SERt3pp:flux 0.0
SERTRS:flux 0.0
SERTRS2:flux 0.0
TRPS1:flux 5.69299883708285e-05
TRPS2:flux 0.0
PROTEIN:flux 0.0


#### This moves the flux to another reaction, GHMT2r, which also breaks down serine. Will continue to knock out these kinds of formulas

In [15]:
model.reactions.GHMT2r.reaction

'ser_L_c + thf_c <=> gly_c + h2o_c + mlthf_c'

In [16]:
with model:
    model.reactions.SERD_L.bounds=0.01,0.01
    model.reactions.GHMT2r.bounds=0.01,0.01
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('ser_L_c','reaction'):
        print("%s:flux %s" % (x.id, x.flux))    

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
SERAT:flux 0.019658420069773808
GPDDA3:flux 0.0
GHMT2r:flux 0.01
PSERT:flux 0.3892699999999998
PSP_L:flux 0.3892699999999998
SERD_L:flux 0.01
PSERtex:flux 0.0
SERabcpp:flux 0.0
SERt3pp:flux 0.34955464994185514
SERTRS:flux 0.0
SERTRS2:flux 0.0
TRPS1:flux 5.69299883708285e-05
TRPS2:flux 0.0
PROTEIN:flux 0.0


In [17]:
model.reactions.SERt3pp.reaction #export reaction

'h_e + ser_L_c --> h_c + ser_L_e'

In [18]:
with model:
    model.reactions.SERD_L.bounds=0.01,0.01
    model.reactions.GHMT2r.bounds=0.01,0.01
    model.reactions.SERt3pp.bounds=0.01,0.01
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('ser_L_c','reaction'):
        print("%s:flux %s" % (x.id, x.flux))    

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
SERAT:flux 0.35921307001162894
GPDDA3:flux 0.0
GHMT2r:flux 0.01
PSERT:flux 0.3892699999999998
PSP_L:flux 0.3892699999999998
SERD_L:flux 0.01
PSERtex:flux 0.0
SERabcpp:flux 0.0
SERt3pp:flux 0.01
SERTRS:flux 0.0
SERTRS2:flux 0.0
TRPS1:flux 5.69299883708285e-05
TRPS2:flux 0.0
PROTEIN:flux 0.0


In [19]:
model.reactions.SERAT.reaction

'accoa_c + ser_L_c --> acser_c + coa_c'

#### They just keep coming. easier now just to force things through TRPS1 by upregulating

In [20]:
with model:
    model.reactions.SERD_L.bounds=0.01,0.01
    model.reactions.GHMT2r.bounds=0.01,0.01
    model.reactions.SERt3pp.bounds=0.01,0.01
    model.reactions.TRPS1.bounds=0.2,0.2
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('ser_L_c','reaction'):
        print("%s:flux %s" % (x.id, x.flux))    

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
SERAT:flux 0.15926999999999977
GPDDA3:flux 0.0
GHMT2r:flux 0.01
PSERT:flux 0.3892699999999998
PSP_L:flux 0.3892699999999998
SERD_L:flux 0.01
PSERtex:flux 0.0
SERabcpp:flux 0.0
SERt3pp:flux 0.01
SERTRS:flux 0.0
SERTRS2:flux 0.0
TRPS1:flux 0.2
TRPS2:flux 0.0
PROTEIN:flux 0.0


#### It now proceeds through TRPS1, as we desired, but the production rate of psilocybin isn't affected.

In [27]:
with model:
    model.reactions.SERD_L.bounds=0.01,0.01
    model.reactions.GHMT2r.bounds=0.01,0.01
    model.reactions.SERt3pp.bounds=0.01,0.01
    model.reactions.TRPS1.bounds=0.2,0.2  
    model.objective = target
    print("Tryptophan reactions")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("TRPTRS [mmol gDW^-1 h^-1]:",model.reactions.TRPTRS.flux)
    print("TRPS2 [mmol gDW^-1 h^-1]:",model.reactions.TRPS2.flux)
    print("CrTdc [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("TRPS1 [mmol gDW^-1 h^-1]:",model.reactions.TRPS1.flux)
    print("TRPt2rpp [mmol gDW^-1 h^-1]:",model.reactions.TRPt2rpp.flux)
    print("TRPabcpp [mmol gDW^-1 h^-1]:",model.reactions.TRPabcpp.flux)
    print("PROTEIN [mmol gDW^-1 h^-1]:",model.reactions.PROTEIN.flux)

Tryptophan reactions
Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPTRS [mmol gDW^-1 h^-1]: 0.0
TRPS2 [mmol gDW^-1 h^-1]: 0.0
CrTdc [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPS1 [mmol gDW^-1 h^-1]: 0.2
TRPt2rpp [mmol gDW^-1 h^-1]: -0.19994307001162917
TRPabcpp [mmol gDW^-1 h^-1]: 0.0
PROTEIN [mmol gDW^-1 h^-1]: 0.0


In [28]:
model.reactions.TRPt2rpp.reaction

'h_e + trp_L_e <-- h_c + trp_L_c'

#### The export reaction is doing weird things and limiting bounds does not help

In [29]:
with model:
    model.reactions.SERD_L.bounds=0.01,0.01
    model.reactions.GHMT2r.bounds=0.01,0.01
    model.reactions.SERt3pp.bounds=0.01,0.01
    model.reactions.TRPS1.bounds=0.2,0.2
    model.objective = target
    print("every reaction going toward psilocybin")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("TRPTRS [mmol gDW^-1 h^-1]:",model.reactions.TRPTRS.flux)
    print("TRPS2 [mmol gDW^-1 h^-1]:",model.reactions.TRPS2.flux)
    print("CrTdc [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("TRPS1 [mmol gDW^-1 h^-1]:",model.reactions.TRPS1.flux)
    print("TRPt2rpp [mmol gDW^-1 h^-1]:",model.reactions.TRPt2rpp.flux)
    print("TRPabcpp [mmol gDW^-1 h^-1]:",model.reactions.TRPabcpp.flux)
    print("PROTEIN [mmol gDW^-1 h^-1]:",model.reactions.PROTEIN.flux)

every reaction going toward psilocybin
Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPTRS [mmol gDW^-1 h^-1]: 0.0
TRPS2 [mmol gDW^-1 h^-1]: 0.0
CrTdc [mmol gDW^-1 h^-1]: 5.69299883708285e-05
TRPS1 [mmol gDW^-1 h^-1]: 0.2
TRPt2rpp [mmol gDW^-1 h^-1]: -0.19994307001162917
TRPabcpp [mmol gDW^-1 h^-1]: 0.0
PROTEIN [mmol gDW^-1 h^-1]: 0.0


In [30]:
model.reactions.TRPS1.reaction

'3ig3p_c + ser_L_c --> g3p_c + h2o_c + trp_L_c'

The tryptophan synthesis reaction involves both serine and C'-(3-Indolyl)-glycerol 3-phosphate. Downregulating other reactions in the serine pathway doesn't seem to help the rate, and upregulating creates issues. Therefore, it is reasoned that the C'-(3-Indolyl)-glycerol 3-phosphate has production rate limits.

In [33]:
model.reactions.query("3ig3p_c","reaction")

[<Reaction IGPS at 0x22ef2016860>,
 <Reaction TRPS1 at 0x22ef2025978>,
 <Reaction TRPS3 at 0x22ef2025e48>,
 <Reaction INPSHE at 0x22ef2031550>]

There is no exchange reaction to easily add more, and the involved reactions form a mostly closed loop (as seen below).

## Relevant Output, Glyceraldehyde 3-phosphate

In [24]:
with model: #model output 1
    model.objective = target
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('g3p_c','reaction'):
        print("%s : %s" % (x.id, x.flux))    

Max. psi production [mmol gDW^-1 h^-1]: 5.69299883708285e-05
DRPA : 0.0
DXPS : 0.0
PY5PSH : 0.0
AGPAT120 : 0.0
AGPAT140 : 0.0
AGPAT141 : 0.0
AGPAT160 : 0.0
AGPAT161 : 0.0
AGPAT180 : 0.0
AGPAT181 : 0.0
G3PAT120 : 0.0
G3PAT140 : 0.0
G3PAT141 : 0.0
G3PAT160 : 0.0
G3PAT161 : 0.0
G3PAT180 : 0.0
G3PAT181 : 0.0
LPLIPAL1A180pp : 0.0
FBA : 3.39087
GAPD : 7.20853
TPI : 3.1829846199922467
IG3PS : 0.0
IGPDH : 0.0
TALA : 0.6346753800077526
TKT1 : 0.6346753800077526
TKT2 : 0.439557078805507
IGPS : 0.19511830120224563
TRPS1 : 0.19511830120224563
TRPS3 : 0.0
INPSHE : 0.0


In [25]:
print(model.reactions.IGPS.name,model.reactions.IGPS.bounds,model.reactions.IGPS.reaction)
print(model.reactions.TRPS1.name,model.reactions.TRPS1.bounds,model.reactions.TRPS1.reaction)
print(model.reactions.TRPS3.name,model.reactions.TRPS3.bounds,model.reactions.TRPS3.reaction)

indole-3-glyceroL_phosphate synthase (0.0, 0.22043) 2cpr5p_c + h_c --> 3ig3p_c + co2_c + h2o_c
tryptophan synthase (indoleglycerol phosphate) (0.0, 0.22043) 3ig3p_c + ser_L_c --> g3p_c + h2o_c + trp_L_c
tryptophan synthase (indoleglycerol phosphate) (0.0, 0.22043) 3ig3p_c --> g3p_c + indole_c


These three reactions form a loop along with the C'-(3-Indolyl)-glycerol 3-phosphate